In [1]:
from unstructured.partition.pdf import partition_pdf
from pathlib import Path
import pickle

file_path = '../Data/Input/' + 'fmea_handbook.pdf'
output_path = '../Data/Output/'


output_dir = Path(output_path)
output_dir.mkdir(exist_ok=True)
output_file_path = output_dir / "chunks.pkl"

chunks = None

OLLAMA_URL = "http://127.0.0.1:11434"

ModuleNotFoundError: No module named 'unstructured'

In [2]:

# Reference: https://docs.unstructured.io/open-source/core-functionality/chunking
chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables

    extract_image_block_types=["Image"],   # Add 'Table' to list to extract image of tables
    # image_output_dir_path=output_path,   # if None, images and tables will saved in base64

    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage

    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,

    # extract_images_in_pdf=True,          # deprecated
)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [4]:
if chunks is not None and len(chunks) > 0:
    with open(output_file_path, "wb") as f:
        pickle.dump(chunks, f)

In [5]:
with open(output_file_path, "rb") as f:
    chunks = pickle.load(f)

In [6]:
set([str(type(el)) for el in chunks])

{"<class 'unstructured.documents.elements.CompositeElement'>"}

In [7]:
# separate tables from texts
tables = []
texts = []

for chunk in chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)

    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

In [8]:
# Get the images from the CompositeElement objects
def get_images_base64(chunks):
    images_b64 = []
    for chunk in chunks:
        if "CompositeElement" in str(type(chunk)):
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                if "Image" in str(type(el)):
                    images_b64.append(el.metadata.image_base64)
    return images_b64

images = get_images_base64(chunks)